In [1]:
!pip install PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 4.3 MB/s eta 0:00:00m eta 0:00:010:00:01


In [2]:
import fitz
import pandas as pd
import numpy as np

class PDF():
    def __init__(self, file):
        self.pdf_doc = fitz.open(file)
    @staticmethod
    def rect_to_svg_line(x1, y1, x2, y2, fill, stroke):
        width = x2 - x1
        height = y2 - y1

        svg_rect = f'<rect x="{x1}" y="{y1}" width="{width}" height="{height}" fill="{fill}" stroke="{stroke}" />\n'
        return svg_rect
    @staticmethod
    def merge_bins(arr):
        result = []
        n = len(arr)
        i = 0
        while i < n:
            if i + 1 < n and arr[i + 1] - arr[i] < 4:
                result.append((arr[i + 1] + arr[i]) / 2)
                i += 2
            else:
                result.append(arr[i])
                i += 1
        result = np.array(result)
        return result
    def get_layout(self, page, skip_rects=0, fill="white", stroke="black"):
        pdf_page = self.pdf_doc[int(page)]
        mediabox = pdf_page.mediabox
        width = mediabox.x1 - mediabox.x0
        height = mediabox.y1 - mediabox.y0
        svg_lines = ''
        for rect in pdf_page.get_drawings()[skip_rects:]:
            x1, y1, x2, y2 = rect['rect'][0], rect['rect'][1], rect['rect'][2], rect['rect'][3]
            svg_lines += self.rect_to_svg_line(x1, y1, x2, y2, fill, stroke)
        svg_content = f'<svg xmlns="http://www.w3.org/2000/svg" width="{width}" height="{height}" viewBox="0 0 {width} {height}">\n' \
          f'<rect x="0" y="0" width="{width}" height="{height}" fill="white" />\n' \
          f'{svg_lines}' \
          f'</svg>'
        return svg_content

    def multipage_table(self, pages, headers, skip_rows=0, skip_cols=0, skip_rects=0):
        df = pd.DataFrame()
        r_ind=-1
        for p in pages:
            pdf_page = self.pdf_doc[int(p)]

            x1_values = []
            y1_values = []
            for rect in pdf_page.get_drawings()[skip_rects:]:
                x1_values.append(rect['rect'][0])
                y1_values.append(rect['rect'][1])
            x_bins = np.sort(list(set(x1_values)))
            y_bins = np.sort(list(set(y1_values)))
            y_grid = self.merge_bins(y_bins)
            x_grid = self.merge_bins(x_bins)

            for row in range(skip_rows, len(y_grid)-1):
                r_ind+=1
                y1, y2 = y_grid[row],y_grid[row+1]

                for col in range(skip_cols, len(x_grid)-1):
                    x1, x2 = x_grid[col],x_grid[col+1]
                    cell_text = pdf_page.get_text("text", clip=(x1, y1, x2, y2))  # Extract text within cell boundaries
                    df.loc[r_ind,headers[col]] = cell_text.replace("\n","")
        return df

In [10]:
from IPython.display import SVG

page_number = 39
columns=[
    "<1>", "<2>", "<3>", 
    "<4>", "<5>", "<6>", 
    "<7>", "<8>", "<9>",
    "<10>", "<11>",
    "<12>",
    "<13>", "<14>",
    "<15>"
]
file = 'Input_Documents/Ambit_HDFC Life_FY23.pdf'
pdf_file = PDF(file)
svg_content = pdf_file.multipage_table(pdf_file,headers=[""], skip_rects=1)
SVG(svg_content)

# If the format is good you can proceed to extract tables
# pages = np.linspace(39,56,18)
# df = pdf_file.multipage_table(pages, headers=columns, skip_rows=4, skip_cols=0, skip_rects=1)
# df.to_csv("out.csv", index=False)

TypeError: 'PDF' object is not iterable

In [5]:
pdf_file.page

AttributeError: 'PDF' object has no attribute 'page'